In [15]:
import os
import time
import random
import numpy as np
from rouge_score import rouge_scorer
import utils
import torch
import transformers
from torch.utils.data import Dataset
import openai
from dotenv import load_dotenv
from async_api import OpenAIMultiClient

In [2]:
load_dotenv(override=True)
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
test_tasks = [
    {
        "instruction": "Suggest a completion for the following python code.\nimport json\n\ndef read_task_from_jsonl(data_file):\n    '''This function will read a .jsonl file and return the ``task`` fields in all the lines.'''",
        "output": "import json\n\ndef read_task_from_jsonl(data_file):\n    '''This function will read a .jsonl file and return the ``task`` fields in all the lines.'''\n    with open(data_file) as fin:\n        return [json.loads(line)['task'] for line in fin]"
    },
    {
        "instruction": "Using the comment as a guide, replace the \"TODO\" comment in the code with the proper code.\ndef greet_to(name):\n    \"\"\"Print a greeting to the name provided.\"\"\"\n    # TODO: assert name is a string\n    print(\"Hello, \" + name + \"!\")",
        "output": "def greet_to(name):\n    \"\"\"Print a greeting to the name provided.\"\"\"\n    # assert name is a string\n    assert isinstance(name, str), \"name must be a string\"\n    print(\"Hello, \" + name + \"!\")"
    },
    {
        "instruction": "Generate a correct python program to fulfill the following purpose:\nCount the occurrence of an element x in a list y.",
        "output": "def count_x(x, y):\n    count = 0\n    for i in range(y):\n        if i == x:\n            count += 1\n    return count"
    },
]

#### Comparison

In [22]:
def make_sync_requests():
    responses = []
    for num in range(1, 20):
        while True:
            try:
                completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "user", "content": f"Can you tell me what is {num} * {num}?"}
                    ]
                )
                print(f"sync request {num} complete")
                break
            except openai.error.OpenAIError as e:
                print(e)
                time.sleep(2)
        responses.append({"num": num, "result": completion.choices[0].message.content})
    return responses


def make_async_requests(api):
    for num in range(1, 20):
        api.request(data={
            "messages": [{
                "role": "user",
                "content": f"Can you tell me what is {num} * {num}?"
            }]
        }, metadata={'num': num})

In [24]:
sync_time_start =  time.time()
sync_responses = make_sync_requests()
for result in sync_responses:
    num = result["num"]
    text = result["result"]
    print(f'{num} * {num} = {text}')
sync_duration = time.time() - sync_time_start
print(f"sync 10 requests took {sync_duration:.2f}s")

async_time_start = time.time()
api = OpenAIMultiClient(endpoint="chats", data_template={"model": "gpt-3.5-turbo"})
api.run_request_function(make_async_requests, api)
for result in api:
    num = result.metadata["num"]
    text = result.response.choices[0].message.content
    print(f'{num} * {num} = {text}')
async_duration = time.time() - async_time_start
print(f"async 10 requests took {async_duration:.2f}s")

sync request 1 complete
sync request 2 complete
sync request 3 complete
sync request 4 complete
sync request 5 complete
sync request 6 complete
sync request 7 complete
sync request 8 complete
sync request 9 complete
sync request 10 complete
sync request 11 complete
sync request 12 complete
sync request 13 complete
sync request 14 complete
sync request 15 complete
sync request 16 complete
sync request 17 complete
sync request 18 complete
sync request 19 complete
1 * 1 = The product of 1 multiplied by 1 is 1.
2 * 2 = The product of 2 multiplied by 2 is 4.
3 * 3 = 3 multiplied by 3 equals 9.
4 * 4 = The product of 4 multiplied by 4 is 16.
5 * 5 = 5 times 5 equals 25.
6 * 6 = The product of 6 multiplied by 6 is 36.
7 * 7 = The product of 7 multiplied by 7 is 49.
8 * 8 = Certainly! 8 multiplied by 8 is equal to 64.
9 * 9 = The product of 9 multiplied by 9 is 81.
10 * 10 = The multiplication of 10 multiplied by 10 is equal to 100.
11 * 11 = Yes, 11 multiplied by 11 equals 121.
12 * 12 = The 

Error processing Payload(endpoint='chats', data={'model': 'gpt-3.5-turbo', 'messages': [{'role': 'user', 'content': 'Can you tell me what is 19 * 19?'}]}, metadata={'num': 19}, max_retries=10, retry_multiplier=1, retry_max=60, attempt=1, failed=False, response=None, callback=None)
Traceback (most recent call last):
  File "/home/nickrosh/evol-teacher/async_api.py", line 111, in _worker
    payload = await self._process_payload(payload)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nickrosh/evol-teacher/async_api.py", line 81, in _process_payload
    payload.response = await openai.ChatCompletion.acreate(**payload.data)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nickrosh/evol-teacher/venv_evol_teacher/lib/python3.11/site-packages/openai/api_resources/chat_completion.py", line 45, in acreate
    return await super().acreate(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nickrosh/evol-teache

Async a 5.3x speedup compared to the synchronous method.

In [28]:
# Now let's do a test evolve generation with the test tasks
def evolve_instructions(instructions, api):
    methods = [
    'Add new constraints and requirements to the original problem, adding approximately 10 additional words.',
    'Replace a commonly used requirement in the programming task with a less common and more specific one.',
    'If the original problem can be solved with only a few logical steps, please add more reasoning steps.',
    'Provide a piece of erroneous code as a reference to increase misdirection.',
    'Propose higher time or space complexity requirements, but please refrain from doing so frequently.'
    ]
    for task in instructions:
        chosen_method = random.choice(methods)
        prompt = f"Please increase the difficulty of the given programming test question a bit.\n\nYou can increase the difficulty using, but not limited to, the following methods:\n{chosen_method}\n\n{task['instruction']}"
        api.request(data={
            "messages": [{
                "role": "user",
                "content": prompt
            }]
        }, metadata={'original_prompt': task['instruction'], 'method': chosen_method})

In [31]:
api = OpenAIMultiClient(
    endpoint="chats",
    concurrency=10,
    max_retries=10,
    wait_interval=2,
    retry_multiplier=1,
    retry_max=60,
    data_template={"model": "gpt-3.5-turbo"},
)
api.run_request_function(evolve_instructions, test_tasks, api)
for result in api:
    original = result.metadata["original_prompt"]
    evol_method = result.metadata["method"]
    new = result.response["choices"][0]["message"]["content"]
    print(f'### ORINGAL PROMPT:\n{original}\n\n')
    print(f'### EVOLVE METHOD:\n{evol_method}\n\n')
    print(f'### NEW PROMPT:\n{new}\n\n')

### ORINGAL PROMPT:
Using the comment as a guide, replace the "TODO" comment in the code with the proper code.
def greet_to(name):
    """Print a greeting to the name provided."""
    # TODO: assert name is a string
    print("Hello, " + name + "!")


### EVOLVE METHOD:
Add new constraints and requirements to the original problem, adding approximately 10 additional words.


### NEW PROMPT:
def greet_to(name):
    """Print a greeting to the name provided."""
    assert isinstance(name, str), "Name should be a string" # Asserting that name is a string
    print("Hello, " + name + "!")


### ORINGAL PROMPT:
Suggest a completion for the following python code.
import json

def read_task_from_jsonl(data_file):
    '''This function will read a .jsonl file and return the ``task`` fields in all the lines.'''


### EVOLVE METHOD:
Add new constraints and requirements to the original problem, adding approximately 10 additional words.


### NEW PROMPT:
import json

def read_task_from_jsonl(data_fil

In [34]:
api = OpenAIMultiClient(
    endpoint="chats",
    concurrency=10,
    max_retries=10,
    wait_interval=2,
    retry_multiplier=1,
    retry_max=60,
    data_template={"model": "gpt-3.5-turbo"},
)
api.run_request_function(evolve_instructions, test_tasks, api)
for result in api:
    original = result.metadata["original_prompt"]
    evol_method = result.metadata["method"]
    new = result.response["choices"][0]["message"]["content"]
    print(f'### ORINGAL PROMPT:\n{original}\n\n')
    print(f'### EVOLVE METHOD:\n{evol_method}\n\n')
    print(f'### NEW PROMPT:\n{new}\n\n')

### ORINGAL PROMPT:
Using the comment as a guide, replace the "TODO" comment in the code with the proper code.
def greet_to(name):
    """Print a greeting to the name provided."""
    # TODO: assert name is a string
    print("Hello, " + name + "!")


### EVOLVE METHOD:
If the original problem can be solved with only a few logical steps, please add more reasoning steps.


### NEW PROMPT:
def greet_to(name):
    """Print a greeting to the name provided."""
    # Add more reasoning steps to validate name is a string
    assert isinstance(name, str), "Name should be a string" 

    print("Hello, " + name + "!")


### ORINGAL PROMPT:
Suggest a completion for the following python code.
import json

def read_task_from_jsonl(data_file):
    '''This function will read a .jsonl file and return the ``task`` fields in all the lines.'''


### EVOLVE METHOD:
If the original problem can be solved with only a few logical steps, please add more reasoning steps.


### NEW PROMPT:
import json

def read_